In [1]:
#!pip install import_ipynb
import import_ipynb
import matplotlib.pyplot as plt
import pandas as pd
from FULL_DATA import final_df

importing Jupyter notebook from FULL_DATA.ipynb
importing Jupyter notebook from Create_Labels.ipynb
(6383, 4)
4377
(4402, 7)
(4377, 7)
(4437, 7)


In [34]:
print('Label Counts:')
final_df['SENTIMENT'] = [0 if (x > 1 and x<3.4999) else 1 if (x > 3.5 and x<4.49999) else -1  for x in final_df['ratings']]
print(final_df['SENTIMENT'].value_counts())
print(final_df.shape)

print('unique companies:',len(final_df['TICKER'].unique()))
final_df.head()

Label Counts:
 1    2367
 0    1144
-1     866
Name: SENTIMENT, dtype: int64
(4377, 9)
unique companies: 997


,TRANSCRIPTS,TICKER,TRANSCRIPT_DATE,RATINGS_DATE,ratings,DATE_DIFF,KEY,SENTIMENT,TOKENIZED_TRANSCRIPT
0,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-07-31,2018-09-28,4.714,59 days,A_2018-09-28,-1,"[image, source, the, motley, fool, agilent, te..."
1,"['Image source: The Motley Fool.', 'Agilent Te...",A,2018-11-19,2018-12-31,4.571,42 days,A_2018-12-31,-1,"[image, source, the, motley, fool, agilent, te..."
2,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-01-31,2019-03-29,5.000,57 days,A_2019-03-29,-1,"[image, source, the, motley, fool, agilent, te..."
3,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-05-14,2019-06-28,4.867,45 days,A_2019-06-28,-1,"[image, source, the, motley, fool, agilent, te..."
4,"['Image source: The Motley Fool.', 'Agilent Te...",A,2019-08-14,2019-09-30,4.286,47 days,A_2019-09-30,1,"[image, source, the, motley, fool, agilent, te..."


In [33]:
final_df = final_df.reset_index(drop=True)

#final_df.to_csv('final_df.csv', index=False)
final_df["TRANSCRIPT_DATE"].min()
#final_df["TRANSCRIPT_DATE"].max()

Timestamp('2017-09-30 00:00:00')

In [4]:
from nltk.tokenize import regexp_tokenize
import string
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """
def tokenize(text, pattern = default_pattern):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return regexp_tokenize(text, pattern)
final_df['TOKENIZED_TRANSCRIPT'] = final_df.apply(lambda row: tokenize(row['TRANSCRIPTS']), axis=1)

In [29]:
import collections
freq_dict = collections.defaultdict(int)
for i in range(len(final_df["TOKENIZED_TRANSCRIPT"])):

    text = final_df["TOKENIZED_TRANSCRIPT"][i]
    for w in text:
        freq_dict[w] += 1

In [6]:
print(sum(freq_dict.values()))

39000201


In [8]:
### Sentiment dictionary related models
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
import string
punctuation_list = list(string.punctuation)

document = pd.read_csv('LoughranMcDonald_MasterDictionary_2018.csv')
negatives = document[document["Negative"] != 0]
positives = document[document["Positive"] != 0]
uncertains = document[document["Uncertainty"] != 0]
litigious = document[document["Litigious"] != 0]

negative_word_list = list(negatives["Word"].str.lower())
positive_word_list = list(positives["Word"].str.lower())
uncertain_word_list = list(uncertains["Word"].str.lower())
litigious_word_list = list(litigious["Word"].str.lower())

column_list = ['KEY','TICKER', 'TRANSCRIPT_DATE','negative_count', 'positive_count', 
               'uncertain_count', 'litigious_count','SENTIMENT']
sentiment_word_counts = pd.DataFrame(columns = column_list)

for i in range(len(final_df["TRANSCRIPTS"])):
    text = final_df["TRANSCRIPTS"][i]
    tokens = word_tokenize(text)
    filtered_tokens=[]
    negative_count = 0
    positive_count = 0
    uncertain_count = 0
    litigious_count = 0
    for w in tokens:
        w = w.lower()
        if w not in stop_words and w not in punctuation_list:
            filtered_tokens.append(w)
            if w in negative_word_list:
                negative_count += 1
            if w in positive_word_list:
                positive_count += 1
            if w in uncertain_word_list:
                uncertain_count += 1
            if w in litigious_word_list:
                litigious_count += 1
    word_counts_row = pd.DataFrame([[final_df.iloc[i]["KEY"], final_df.iloc[i]["TICKER"],final_df.iloc[i]["TRANSCRIPT_DATE"], 
                                    negative_count, positive_count, uncertain_count, litigious_count,
                                    final_df.iloc[i]['SENTIMENT']]], columns = column_list)
    
    sentiment_word_counts = sentiment_word_counts.append(word_counts_row)
sentiment_word_counts        

,KEY,TICKER,TRANSCRIPT_DATE,negative_count,positive_count,uncertain_count,litigious_count,SENTIMENT
0,A_2018-09-28,A,2018-07-31,96,198,101,4,-1
0,A_2018-12-31,A,2018-11-19,98,215,109,6,-1
0,A_2019-03-29,A,2019-01-31,86,208,110,3,-1
0,A_2019-06-28,A,2019-05-14,119,143,121,6,-1
0,A_2019-09-30,A,2019-08-14,76,173,89,8,1
0,A_2019-12-31,A,2019-11-25,84,172,127,5,1
0,AA_2018-09-28,AA,2018-06-30,178,91,77,22,1
0,AA_2019-03-29,AA,2019-01-16,145,144,72,16,1
0,AA_2019-06-28,AA,2019-04-17,145,189,94,11,1
0,AA_2019-09-30,AA,2019-07-17,165,184,73,12,1


In [11]:
#sentiment_word_counts.to_csv('sentiment_word_counts.csv', index = False)

In [ ]:
#sentiment_word_counts = pd.read_csv('sentiment_word_counts.csv')

In [27]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

X = sentiment_word_counts[["negative_count","positive_count","uncertain_count", "litigious_count"]]
Y = sentiment_word_counts['SENTIMENT']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)
Y_test = Y_test.to_numpy()
lm_results = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})
print(lm_results.head(10)) 

rmse = np.sqrt(mean_squared_error(Y_test, y_pred))      
r2_value = r2_score(Y_test, y_pred)                     

print("Intercept: \n", model.intercept_)
print("Root Mean Square Error \n", rmse)
print("R^2 Value: \n", r2_value)

X = sentiment_word_counts[["negative_count","positive_count","uncertain_count", "litigious_count"]]
Y = np.asarray(sentiment_word_counts['SENTIMENT'],dtype=np.float64)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

clf = linear_model.LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
logreg_model = clf.fit(X_train, Y_train)
y_pred_prob = logreg_model.predict_proba(X_test)
y_pred = np.argmax(y_pred_prob, axis = 1)
y_pred[y_pred == 0] = -1
y_pred[y_pred == 1] = 0
y_pred[y_pred == 2] = 1


logreg_results = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})
print(logreg_results.head(10)) 
print(logreg_results['Predicted'].value_counts())

print('Number correct:', str(correct))
print('Multinomial Logistic Regression Accuracy: ', accuracy_score(Y_test, y_pred))
print('Multinomial Logistic Regression F1 Score: ', f1_score(Y_test, y_pred, average = 'macro'))

  Actual  Predicted
0     -1   0.335117
1      1   0.316071
2      1   0.229167
3      1   0.319982
4      1   0.339885
5     -1   0.393025
6      1   0.190602
7      0   0.499733
8     -1   0.337927
9      1   0.307090
Intercept: 
 0.037399206419355846
Root Mean Square Error 
 0.7874337125067894
R^2 Value: 
 0.011901488405139937
   Actual  Predicted
0    -1.0          1
1     1.0          1
2     1.0          1
3     1.0          1
4     1.0          1
5    -1.0          1
6     1.0          1
7     0.0          1
8    -1.0          1
9     1.0          1
1    867
0      9
Name: Predicted, dtype: int64
Number correct: 357
Multinomial Logistic Regression Accuracy:  0.5536529680365296
Multinomial Logistic Regression F1 Score:  0.25172132484907817


C:\Users\Frank\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)
multi_nb_pred = clf.predict(X_test)

multi_nb_results = pd.DataFrame({'Actual': Y_test, 'Predicted': multi_nb_pred})
print(multi_nb_results.head(10))
print(multi_nb_results['Predicted'].value_counts())

correct = np.sum(multi_nb_pred == Y_test)
print('Number correct:', str(correct))
print('Multinomial Naive Bayes Accuracy: ', correct/len(multi_nb_pred))
print('Multinomial Naive Bayes F1 Score: ', f1_score(Y_test, y_pred, average = 'macro'))

   Actual  Predicted
0    -1.0        1.0
1     1.0       -1.0
2     1.0        0.0
3     1.0        0.0
4     1.0       -1.0
5    -1.0        0.0
6     1.0        1.0
7     0.0        0.0
8    -1.0        0.0
9     1.0        1.0
 1.0    507
 0.0    248
-1.0    121
Name: Predicted, dtype: int64
Number correct: 357
Multinomial Naive Bayes Accuracy:  0.4075342465753425
Multinomial Naive Bayes F1 Score:  0.25172132484907817


C:\Users\Frank\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
